# Importing the libraries

In [1]:
import numpy as np
import random
import os
import tensorflow as tf
from tensorflow import keras
import platform
from tensorflow.keras import layers, optimizers, losses, models, Input, Model
import time # Per misurare il tempo di training
from tensorflow.keras.callbacks import EarlyStopping # Per l'early stopping
import matplotlib.pyplot as plt
from tqdm import tqdm # Per mostrare una barra di progresso

# Set seeds for random operations.

In [2]:
# --- 1. Impostazione del Seed Globale all'inizio del tuo script ---
# Questo è il punto chiave per la riproducibilità di TUTTO ciò che segue.
MASTER_RANDOM_SEED = 42
np.random.seed(MASTER_RANDOM_SEED)
random.seed(MASTER_RANDOM_SEED) # Imposta anche il seed per la libreria 'random' di Python se la usi
tf.random.set_seed(MASTER_RANDOM_SEED)
os.environ['PYTHONHASHSEED'] = str(MASTER_RANDOM_SEED) # Per operazioni basate su hash (es. ordine dei dizionari)
os.environ['TF_DETERMINISTIC_OPS'] = '1' # Forza operazioni deterministiche in TensorFlow 2.x

# Print the HW Specs.

In [3]:
print("--- Dettagli dell'Architettura Hardware della Sessione Colab ---\n")

# --- 1. Dettagli CPU ---
print("--- Dettagli CPU ---")
!lscpu
print("\n")

# --- 2. Dettagli RAM (Memoria) ---
print("--- Dettagli RAM (Memoria) ---")
!cat /proc/meminfo | grep MemTotal
print("\n")

# --- 3. Dettagli Spazio su Disco ---
print("--- Dettagli Spazio su Disco ---")
!df -h /
print("\n")

# --- 4. Dettagli Acceleratore Hardware (GPU/TPU) ---
print("--- Dettagli Acceleratore Hardware (GPU/TPU) ---")
try:
    tpu_address = os.environ.get('COLAB_TPU_ADDR')
    if tpu_address:
        resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu=tpu_address)
        tf.config.experimental_connect_to_cluster(resolver)
        tf.tpu.experimental.initialize_tpu_system(resolver)
        print(f"Tipo Acceleratore: TPU (indirizzo: {tpu_address})")
        print("Dispositivi TPU disponibili:")
        for device in tf.config.list_logical_devices('TPU'):
            print(f"  - {device.name}")
    else:
        gpus = tf.config.list_physical_devices('GPU')
        if gpus:
            print(f"Tipo Acceleratore: GPU")
            for gpu in gpus:
                print(f"  - Dispositivo GPU rilevato: {gpu.name}")
            print("\nDettagli GPU specifici (da `!nvidia-smi`):")
            !nvidia-smi
        else:
            print("Tipo Acceleratore: Nessuna GPU o TPU rilevata (in uso CPU)")

except Exception as e:
    print(f"Si è verificato un errore durante la rilevazione dell'acceleratore: {e}")
    print("Tentativo di rilevare i dispositivi TensorFlow standard:")
    devices = tf.config.list_logical_devices()
    if devices:
        for device in devices:
            print(f"  - Dispositivo rilevato: {device.name}, Tipo: {device.device_type}")
    else:
        print("Nessun dispositivo TensorFlow rilevato.")

print("\n--- Analisi Dettagli Hardware Completata ---")

--- Dettagli dell'Architettura Hardware della Sessione Colab ---

--- Dettagli CPU ---
Architecture:             x86_64
  CPU op-mode(s):         32-bit, 64-bit
  Address sizes:          46 bits physical, 48 bits virtual
  Byte Order:             Little Endian
CPU(s):                   2
  On-line CPU(s) list:    0,1
Vendor ID:                GenuineIntel
  Model name:             Intel(R) Xeon(R) CPU @ 2.00GHz
    CPU family:           6
    Model:                85
    Thread(s) per core:   2
    Core(s) per socket:   1
    Socket(s):            1
    Stepping:             3
    BogoMIPS:             4000.38
    Flags:                fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge m
                          ca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht sysc
                          all nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xt
                          opology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq
                           ssse3 fma cx16 pcid sse4_1 sse4

# Connect To Gdrive to store the datasets created.

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Define the paths

In [ ]:
# Percorsi dataset
paths = {
    "0-20": {
        "train": "/content/drive/MyDrive/GitHub/AWGN/dataset/training_0-20_SNR_10_BETA.npz",
        "val": "/content/drive/MyDrive/GitHub/AWGN/dataset/validation_0-20_SNR_10_BETA.npz",
    },
    "11-15": {
        "train": "/content/drive/MyDrive/GitHub/AWGN/dataset/training_11-15_SNR_10_BETA.npz",
        "val": "/content/drive/MyDrive/GitHub/AWGN/dataset/validation_11-15_SNR_10_BETA.npz",
    }
}

# Directory salvataggio modelli
save_dir = "/content/drive/MyDrive/GitHub/AWGN/trained_models/benchmarks"
os.makedirs(save_dir, exist_ok=True)

# Load the datasets.

In [5]:
def load_dataset(filepath):
    """
    Carica i dati X e y da un file .npz.
    Si aspetta che il file contenga 'X_train' o 'X_val' e 'y_train' o 'y_val'.
    """
    data = np.load(filepath)

    # Controlla se le chiavi per il training sono presenti
    if 'X_train' in data and 'y_train' in data:
        print(f"  Caricato Training data da: {filepath}")
        return data['X_train'], data['y_train']
    # Altrimenti, controlla se le chiavi per la validation sono presenti
    elif 'X_val' in data and 'y_val' in data:
        print(f"  Caricato Validation data da: {filepath}")
        return data['X_val'], data['y_val']
    else:
        # Se nessuna delle combinazioni attese è trovata, solleva un errore
        raise ValueError(f"Il file {filepath} non contiene i dati X e y attesi (né 'X_train'/'y_train' né 'X_val'/'y_val'). "
                         f"Chiavi trovate: {list(data.keys())}")

# Benchmark models definition

In [ ]:
# Funzioni modello

def get_ultracan_v3(input_shape, model_name='ULTRA-CAN-v3_10_BETA'):
    inputs = Input(shape=input_shape)

    # Reshape for Conv1D
    x = layers.Reshape((input_shape[0], 1))(inputs)

    # Lightweight convolutional front-end
    x = layers.Conv1D(16, 3, padding='same', activation='relu')(x)
    x = layers.Conv1D(32, 3, padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)

    # BiGRU backbone (replaces heavy BiLSTM)
    x = layers.Bidirectional(layers.GRU(64, return_sequences=True))(x)

    # Attention mechanism (self-attention)
    query = layers.GlobalAveragePooling1D()(x)
    query = layers.RepeatVector(x.shape[1])(query)
    attention = layers.Multiply()([x, query])
    x = layers.Add()([x, attention])

    # Temporal aggregation
    x = layers.GlobalAveragePooling1D()(x)

    # Classifier
    x = layers.Dense(64, activation='relu')(x)
    x = layers.Dense(32, activation='relu')(x)
    outputs = layers.Dense(2, activation='softmax')(x)

    model = Model(inputs, outputs, name=model_name)
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    return model

def get_ultracan_v2(input_shape, model_name='ULTRA-CAN-v2'):
    inputs = Input(shape=input_shape)
    x = layers.Reshape((input_shape[0], 1))(inputs)

    # Lightweight CNN
    x = layers.Conv1D(8, 3, padding='same', activation='relu')(x)
    x = layers.Conv1D(8, 3, padding='same', activation='relu')(x)

    # Temporal pooling (tipo downsampling trainabile)
    x_pooled = layers.Conv1D(8, 3, strides=2, padding='same', activation='relu')(x)

    # Cross Attention tra media e input
    query = layers.GlobalAveragePooling1D()(x_pooled)         # (None, 8)
    query = layers.RepeatVector(x.shape[1])(query)            # (None, time_steps, 8)
    attn_weights = layers.Multiply()([x, query])              # element-wise attention
    attn = layers.Add()([x, attn_weights])                    # residual connection

    # Final classifier
    x = layers.GlobalAveragePooling1D()(attn)
    x = layers.Dense(32, activation='relu')(x)
    outputs = layers.Dense(2, activation='softmax')(x)

    model = Model(inputs, outputs, name=model_name)
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    return model

def get_ultracan_model(input_shape, model_name='ULTRA-CAN'):
    inputs = Input(shape=input_shape)

    # Reshape per conv1D
    x = layers.Reshape((input_shape[0], 1), name="reshape")(inputs)

    # Leggera CNN
    x = layers.Conv1D(8, kernel_size=3, padding='same', activation='relu', name="cnn1")(x)
    x = layers.Conv1D(8, kernel_size=3, padding='same', activation='relu', name="cnn2")(x)

    # Temporal convolution (tipo TCN semplificato con dilatazione)
    x = layers.Conv1D(8, kernel_size=3, dilation_rate=2, padding='same', activation='relu', name="tcn1")(x)

    # Attention leggera (tipo global self-attention)
    attention = layers.Permute((2,1), name="permute_for_attention")(x)
    attention = layers.Dense(input_shape[0], activation='softmax', name="attention_weights")(attention)
    attention = layers.Permute((2,1), name="unpermute")(attention)
    x = layers.Multiply(name="apply_attention")([x, attention])

    # Pooling e FC finale
    x = layers.GlobalAveragePooling1D(name="global_avg_pool")(x)
    x = layers.Dense(32, activation='relu', name="fc1")(x)
    outputs = layers.Dense(2, activation='softmax', name="output")(x)

    model = Model(inputs, outputs, name=model_name)
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    return model

def get_tdnn_model(input_shape, model_name='TDNN OFDM-DCSK'):
    # Dal paper: "Reliable and Secure Deep Learning-Based OFDM-DCSK Transceiver Design Without Delivery of Reference Chaotic Sequences"
    inputs = Input(shape=input_shape)
    input_length = input_shape[0]
    x = layers.Reshape((input_length, 1), name="reshape_for_conv")(inputs)
    x = layers.Conv1D(4, 2, activation='relu', padding='same', name="tdnn_conv1")(x)
    x = layers.Conv1D(8, 4, activation='relu', padding='same', name="tdnn_conv2")(x)
    x = layers.Conv1D(16, 8, activation='relu', padding='same', name="tdnn_conv3")(x)
    x = layers.Bidirectional(layers.LSTM(64, return_sequences=False), name="bidirectional_lstm")(x)
    x = layers.Dense(128, name="fc1")(x)
    x = layers.BatchNormalization(name="batch_norm")(x)
    x = layers.Activation('relu', name="relu_fc1")(x)
    outputs = layers.Dense(2, activation='softmax', name="output_softmax")(x)
    model = Model(inputs, outputs, name='DNN_Aided_Demodulator')
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

def get_mc_dlcs_model(input_shape, model_name='MC-DLCSK'):
    # Dal paper: "A Multi-Carrier Deep Learning CSK Communication System"
    if input_shape[0] % 2 != 0:
        raise ValueError("input_shape deve essere pari per MC-DLCSK")
    inputs = Input(shape=input_shape)
    x = layers.Reshape((input_shape[0], 1), name="reshape_for_lstm")(inputs)
    x = layers.Bidirectional(layers.LSTM(50, return_sequences=False), name="bidirectional_lstm")(x)
    x = layers.Dense(64, activation='relu', name="fc1_relu")(x)
    outputs = layers.Dense(2, activation='softmax', name="output_softmax")(x)
    model = Model(inputs, outputs, name=model_name)
    model.compile(optimizer=tf.keras.optimizers.Adam(0.01), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

def get_lstm_dnn_model(input_shape, model_name='LSTM_DNN_OFDM_DCSK'):
    # Dal paper: "Intelligent and Reliable Deep Learning LSTM Neural Networks-Based OFDM-DCSK Demodulation Design"
    inputs = Input(shape=input_shape)
    if len(input_shape) == 1:
        x = layers.Reshape((1, input_shape[0]))(inputs)
    elif len(input_shape) == 2:
        x = inputs
    else:
        raise ValueError("input_shape deve essere 1D o 2D")
    x = layers.LSTM(19, return_sequences=True)(x)
    x = layers.TimeDistributed(layers.Dense(38, activation='relu'))(x)
    x = layers.TimeDistributed(layers.BatchNormalization())(x)
    x = layers.GlobalAveragePooling1D()(x)
    outputs = layers.Dense(2, activation='softmax')(x)
    model = Model(inputs, outputs, name=model_name)
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

def get_ultra_can_mini(input_shape):
    inputs = Input(shape=input_shape)

    # Conv1D più piccola (kernel su sequenze brevissime)
    x = layers.Reshape((input_shape[0], 1))(inputs)  # Shape: (5,1)
    x = layers.Conv1D(8, kernel_size=2, activation='relu', padding='same')(x)
    x = layers.Conv1D(16, kernel_size=2, activation='relu', padding='same')(x)

    # Global max pooling su tutta la sequenza (pochi timestep)
    x = layers.GlobalMaxPooling1D()(x)

    # Dense ridotte
    x = layers.Dense(32, activation='relu')(x)
    x = layers.Dense(16, activation='relu')(x)

    outputs = layers.Dense(2, activation='softmax')(x)
    model = Model(inputs, outputs, name="ULTRA_CAN_mini")
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    return model

models_dict = {
    #"ulta-can-mini-5-beta": get_ultra_can_mini,
    "Ultra-CAN-v3-10-BETA": get_ultracan_v3,
    #"TDNN OFDM-DCSK": get_tdnn_model,
    #"MC-DLCSK": get_mc_dlcs_model,
    #"LSTM_DNN_OFDM_DCSK": get_lstm_dnn_model
}

# Training...

# BETA = 5, 10, 15, 20, 30

In [12]:
BETA = [10,15,20,30]

for current_beta in BETA:
    print(current_beta)
    # Percorsi dataset
    paths = {
        "0-20": {
            "train": "/content/drive/MyDrive/GitHub/AWGN/dataset/training_0-20_SNR_"+str(current_beta)+"_BETA.npz",
            "val": "/content/drive/MyDrive/GitHub/AWGN/dataset/validation_0-20_SNR_"+str(current_beta)+"_BETA.npz",
        },
        "11-15": {
            "train": "/content/drive/MyDrive/GitHub/AWGN/dataset/training_11-15_SNR_"+str(current_beta)+"_BETA.npz",
            "val": "/content/drive/MyDrive/GitHub/AWGN/dataset/validation_11-15_SNR_"+str(current_beta)+"_BETA.npz",
        }
    }

    # Directory salvataggio modelli
    save_dir = "/content/drive/MyDrive/GitHub/AWGN/trained_models/benchmarks"
    os.makedirs(save_dir, exist_ok=True)

    def get_ultracan_v3(input_shape, model_name='ULTRA-CAN-v3_'+str(current_beta)+'_BETA'):
        inputs = Input(shape=input_shape)

        # Reshape for Conv1D
        x = layers.Reshape((input_shape[0], 1))(inputs)

        # Lightweight convolutional front-end
        x = layers.Conv1D(16, 3, padding='same', activation='relu')(x)
        x = layers.Conv1D(32, 3, padding='same', activation='relu')(x)
        x = layers.BatchNormalization()(x)

        # BiGRU backbone (replaces heavy BiLSTM)
        x = layers.Bidirectional(layers.GRU(64, return_sequences=True))(x)

        # Attention mechanism (self-attention)
        query = layers.GlobalAveragePooling1D()(x)
        query = layers.RepeatVector(x.shape[1])(query)
        attention = layers.Multiply()([x, query])
        x = layers.Add()([x, attention])

        # Temporal aggregation
        x = layers.GlobalAveragePooling1D()(x)

        # Classifier
        x = layers.Dense(64, activation='relu')(x)
        x = layers.Dense(32, activation='relu')(x)
        outputs = layers.Dense(2, activation='softmax')(x)

        model = Model(inputs, outputs, name=model_name)
        model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

        return model

    models_dict = {
        #"ulta-can-mini-5-beta": get_ultra_can_mini,
        "Ultra-CAN-v3-"+str(current_beta)+"-BETA": get_ultracan_v3,
        #"TDNN OFDM-DCSK": get_tdnn_model,
        #"MC-DLCSK": get_mc_dlcs_model,
        #"LSTM_DNN_OFDM_DCSK": get_lstm_dnn_model
    }

    # Addestramento
    for snr_range, datasets in paths.items():
        x_train, y_train = load_dataset(datasets['train'])
        x_val, y_val = load_dataset(datasets['val'])

        print(x_train.shape)

        for model_name, builder in models_dict.items():
            print(f"\nAddestramento modello: {model_name} | SNR range: {snr_range}")
            model = builder(input_shape=x_train.shape[1:])

            callback = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

            start_time = time.time() # Registra il tempo di inizio training

            model.fit(
                x_train, y_train,
                validation_data=(x_val, y_val),
                epochs=30,
                batch_size=32,
                callbacks=[callback],
                verbose=1
            )

            end_time = time.time() # Registra il tempo di fine training
            total_training_time = end_time - start_time

            print(f"Training completato.")
            print(f"Tempo totale di training: {total_training_time:.2f} secondi.") # Stampa il tempo totale

            save_path = os.path.join(save_dir, f"{model_name.replace(' ', '_')}_awgn_snr_{snr_range}.h5")
            model.save(save_path)
            print(f"Salvato modello in: {save_path}")

10
  Caricato Training data da: /content/drive/MyDrive/GitHub/AWGN/dataset/training_0-20_SNR_10_BETA.npz
  Caricato Validation data da: /content/drive/MyDrive/GitHub/AWGN/dataset/validation_0-20_SNR_10_BETA.npz
(60000, 20)

Addestramento modello: Ultra-CAN-v3-10-BETA | SNR range: 0-20
Epoch 1/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 39s 10ms/step - accuracy: 0.9581 - loss: 0.1141 - val_accuracy: 0.9742 - val_loss: 0.0666
Epoch 2/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 18s 10ms/step - accuracy: 0.9791 - loss: 0.0559 - val_accuracy: 0.9819 - val_loss: 0.0494
Epoch 3/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 21s 10ms/step - accuracy: 0.9809 - loss: 0.0508 - val_accuracy: 0.9815 - val_loss: 0.0476
Epoch 4/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 19s 10ms/step - accuracy: 0.9825 - loss: 0.0473 - val_accuracy: 0.9840 - val_loss: 0.0430
Epoch 5/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 19s 9ms/step - accuracy: 0.9838 - loss: 0.0441 - val_accuracy: 0.9836 - val_loss: 0.0441
Epoch 6/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 20s 9ms/step 

Training completato.
Tempo totale di training: 318.82 secondi.
Salvato modello in: /content/drive/MyDrive/GitHub/AWGN/trained_models/benchmarks/Ultra-CAN-v3-10-BETA_awgn_snr_0-20.h5
  Caricato Training data da: /content/drive/MyDrive/GitHub/AWGN/dataset/training_11-15_SNR_10_BETA.npz
  Caricato Validation data da: /content/drive/MyDrive/GitHub/AWGN/dataset/validation_11-15_SNR_10_BETA.npz
(60000, 20)

Addestramento modello: Ultra-CAN-v3-10-BETA | SNR range: 11-15
Epoch 1/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 22s 10ms/step - accuracy: 0.9814 - loss: 0.0578 - val_accuracy: 0.9989 - val_loss: 0.0032
Epoch 2/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 18s 10ms/step - accuracy: 0.9990 - loss: 0.0032 - val_accuracy: 0.9987 - val_loss: 0.0041
Epoch 3/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 21s 10ms/step - accuracy: 0.9995 - loss: 0.0020 - val_accuracy: 0.9996 - val_loss: 0.0025
Epoch 4/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 20s 10ms/step - accuracy: 0.9995 - loss: 0.0015 - val_accuracy: 0.9984 - val_loss: 0.0057
Epoc

Training completato.
Tempo totale di training: 460.47 secondi.
Salvato modello in: /content/drive/MyDrive/GitHub/AWGN/trained_models/benchmarks/Ultra-CAN-v3-10-BETA_awgn_snr_11-15.h5
15
  Caricato Training data da: /content/drive/MyDrive/GitHub/AWGN/dataset/training_0-20_SNR_15_BETA.npz
  Caricato Validation data da: /content/drive/MyDrive/GitHub/AWGN/dataset/validation_0-20_SNR_15_BETA.npz
(60000, 30)

Addestramento modello: Ultra-CAN-v3-15-BETA | SNR range: 0-20
Epoch 1/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 23s 10ms/step - accuracy: 0.9747 - loss: 0.0729 - val_accuracy: 0.9919 - val_loss: 0.0229
Epoch 2/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 19s 10ms/step - accuracy: 0.9925 - loss: 0.0208 - val_accuracy: 0.9926 - val_loss: 0.0191
Epoch 3/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 19s 10ms/step - accuracy: 0.9937 - loss: 0.0169 - val_accuracy: 0.9924 - val_loss: 0.0183
Epoch 4/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 20s 10ms/step - accuracy: 0.9946 - loss: 0.0149 - val_accuracy: 0.9894 - val_loss: 0.0297
Epo

Training completato.
Tempo totale di training: 283.83 secondi.
Salvato modello in: /content/drive/MyDrive/GitHub/AWGN/trained_models/benchmarks/Ultra-CAN-v3-15-BETA_awgn_snr_0-20.h5
  Caricato Training data da: /content/drive/MyDrive/GitHub/AWGN/dataset/training_11-15_SNR_15_BETA.npz
  Caricato Validation data da: /content/drive/MyDrive/GitHub/AWGN/dataset/validation_11-15_SNR_15_BETA.npz
(60000, 30)

Addestramento modello: Ultra-CAN-v3-15-BETA | SNR range: 11-15
Epoch 1/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 23s 11ms/step - accuracy: 0.9899 - loss: 0.0363 - val_accuracy: 0.9998 - val_loss: 2.4703e-04
Epoch 2/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 19s 10ms/step - accuracy: 0.9998 - loss: 6.0339e-04 - val_accuracy: 0.9996 - val_loss: 0.0011
Epoch 3/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 22s 11ms/step - accuracy: 0.9999 - loss: 3.6203e-04 - val_accuracy: 0.9999 - val_loss: 6.4539e-05
Epoch 4/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 19s 10ms/step - accuracy: 0.9998 - loss: 9.9891e-04 - val_accuracy: 1.0000 - v

Training completato.
Tempo totale di training: 638.20 secondi.
Salvato modello in: /content/drive/MyDrive/GitHub/AWGN/trained_models/benchmarks/Ultra-CAN-v3-15-BETA_awgn_snr_11-15.h5
20
  Caricato Training data da: /content/drive/MyDrive/GitHub/AWGN/dataset/training_0-20_SNR_20_BETA.npz
  Caricato Validation data da: /content/drive/MyDrive/GitHub/AWGN/dataset/validation_0-20_SNR_20_BETA.npz
(60000, 40)

Addestramento modello: Ultra-CAN-v3-20-BETA | SNR range: 0-20
Epoch 1/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 23s 11ms/step - accuracy: 0.9838 - loss: 0.0487 - val_accuracy: 0.9944 - val_loss: 0.0147
Epoch 2/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 20s 11ms/step - accuracy: 0.9954 - loss: 0.0125 - val_accuracy: 0.9952 - val_loss: 0.0129
Epoch 3/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 20s 11ms/step - accuracy: 0.9967 - loss: 0.0091 - val_accuracy: 0.9933 - val_loss: 0.0203
Epoch 4/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 20s 11ms/step - accuracy: 0.9971 - loss: 0.0080 - val_accuracy: 0.9947 - val_loss: 0.0173
Epo

Training completato.
Tempo totale di training: 407.98 secondi.
Salvato modello in: /content/drive/MyDrive/GitHub/AWGN/trained_models/benchmarks/Ultra-CAN-v3-20-BETA_awgn_snr_0-20.h5
  Caricato Training data da: /content/drive/MyDrive/GitHub/AWGN/dataset/training_11-15_SNR_20_BETA.npz
  Caricato Validation data da: /content/drive/MyDrive/GitHub/AWGN/dataset/validation_11-15_SNR_20_BETA.npz
(60000, 40)

Addestramento modello: Ultra-CAN-v3-20-BETA | SNR range: 11-15
Epoch 1/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 25s 11ms/step - accuracy: 0.9886 - loss: 0.0359 - val_accuracy: 1.0000 - val_loss: 9.5866e-06
Epoch 2/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 21s 11ms/step - accuracy: 1.0000 - loss: 2.2363e-06 - val_accuracy: 1.0000 - val_loss: 2.0150e-06
Epoch 3/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 40s 11ms/step - accuracy: 1.0000 - loss: 4.3838e-07 - val_accuracy: 1.0000 - val_loss: 1.4653e-06
Epoch 4/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 20s 11ms/step - accuracy: 1.0000 - loss: 1.2183e-07 - val_accuracy: 1.0000

Training completato.
Tempo totale di training: 923.37 secondi.
Salvato modello in: /content/drive/MyDrive/GitHub/AWGN/trained_models/benchmarks/Ultra-CAN-v3-20-BETA_awgn_snr_11-15.h5
30
  Caricato Training data da: /content/drive/MyDrive/GitHub/AWGN/dataset/training_0-20_SNR_30_BETA.npz
  Caricato Validation data da: /content/drive/MyDrive/GitHub/AWGN/dataset/validation_0-20_SNR_30_BETA.npz
(60000, 60)

Addestramento modello: Ultra-CAN-v3-30-BETA | SNR range: 0-20
Epoch 1/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 28s 13ms/step - accuracy: 0.9891 - loss: 0.0399 - val_accuracy: 0.9979 - val_loss: 0.0064
Epoch 2/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 38s 12ms/step - accuracy: 0.9989 - loss: 0.0029 - val_accuracy: 0.9992 - val_loss: 0.0025
Epoch 3/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 41s 12ms/step - accuracy: 0.9992 - loss: 0.0023 - val_accuracy: 0.9969 - val_loss: 0.0092
Epoch 4/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 40s 11ms/step - accuracy: 0.9996 - loss: 0.0015 - val_accuracy: 0.9985 - val_loss: 0.0039
Epo

Training completato.
Tempo totale di training: 673.00 secondi.
Salvato modello in: /content/drive/MyDrive/GitHub/AWGN/trained_models/benchmarks/Ultra-CAN-v3-30-BETA_awgn_snr_0-20.h5
  Caricato Training data da: /content/drive/MyDrive/GitHub/AWGN/dataset/training_11-15_SNR_30_BETA.npz
  Caricato Validation data da: /content/drive/MyDrive/GitHub/AWGN/dataset/validation_11-15_SNR_30_BETA.npz
(60000, 60)

Addestramento modello: Ultra-CAN-v3-30-BETA | SNR range: 11-15
Epoch 1/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 25s 11ms/step - accuracy: 0.9930 - loss: 0.0355 - val_accuracy: 0.9999 - val_loss: 7.9129e-05
Epoch 2/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 41s 12ms/step - accuracy: 1.0000 - loss: 3.4632e-06 - val_accuracy: 1.0000 - val_loss: 2.5665e-07
Epoch 3/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 21s 11ms/step - accuracy: 1.0000 - loss: 3.1665e-07 - val_accuracy: 1.0000 - val_loss: 6.3314e-08
Epoch 4/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 44s 13ms/step - accuracy: 1.0000 - loss: 6.4889e-08 - val_accuracy: 1.0000

Training completato.
Tempo totale di training: 1119.71 secondi.
Salvato modello in: /content/drive/MyDrive/GitHub/AWGN/trained_models/benchmarks/Ultra-CAN-v3-30-BETA_awgn_snr_11-15.h5
